In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import pg8000

In [2]:
import re

In [3]:
import time

# 1 Using parlament.ch to get all "Fragestunde"-Affairs for Councillors

- It would be nicer to do this through API but the API isn't that sophisticated and the webpage itself is too complicated to scrape with requests, so we use selenium

In [4]:
url = "https://www.parlament.ch/de/ratsbetrieb/suche-curia-vista#k=PdBusinessCouncilorPersonNumber:4030 PdAffairTypeId:14"

In [5]:
driver = webdriver.Chrome()

In [6]:
councillor_id = "4030"

In [22]:
councillor_affairs = []

In [23]:
driver.get(url)

In [9]:
#This appends all councillor_id + affair_id on a page to the list of dctionaries
def scrape_affairs():
    doc = BeautifulSoup(driver.page_source, "html.parser")
    divs = doc.find_all('div', class_='ms-srch-item')
    for div in divs:
        current = {}
        current['councillor_id'] = councillor_id
        current['affair_id'] = div.a['href'].split('=')[1]
        councillor_affairs.append(current)

In [19]:
scrape_affairs()

In [17]:
driver.find_element_by_id("PageLinkNext").click()

In [24]:
while True:
    try:
        time.sleep(10)
        #This gives us a list of dictionaries containing max 10 affair_ids per councillor
        scrape_affairs()
        #We repeat this until no data left
        driver.find_element_by_id("PageLinkNext").click()
    except:
        break

In [ ]:
df = pd.DataFrame(councillor_affairs, columns=['councillor_id', 'affair_id'])
df

In [20]:
len(councillor_affairs)

23

# 2 Save into the database

- Connect to our new database "parl"

In [ ]:
conn = pg8000.connect(database="parl", user="webapp", password="webapp")

In [ ]:
cursor = conn.cursor()

- prepare a function to insert the data and call it on the df

In [ ]:
def insert_councillor(row):
    query = "INSERT INTO councillors (councillor_id, firstName, lastName, canton, party) "
    query += "VALUES ('"+row.councillor_id+"', '"+row.firstName+"', '"+row.lastName+"', '"+row.canton+"', '"+row.party+"');"
    cursor.execute(query)

In [ ]:
df.apply(insert_councillor, axis=1)

In [ ]:
conn.commit()

# 3 Check results

In [ ]:
query2 = "SELECT * FROM councillors;"

In [ ]:
cursor.execute(query2)

In [ ]:
result = cursor.fetchall()

In [ ]:
for row in result:
    print (row)

In [ ]:
conn.close()